In [6]:
from openai import AzureOpenAI
import httpx, requests
import json
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

In [7]:

key_vault_url = "https://user25-tuhin-kv.vault.azure.net/"
credential = DefaultAzureCredential()

client = SecretClient(vault_url=key_vault_url, credential=credential)

secret_name_news_api = "news-api-key"
news_api_key = client.get_secret(secret_name_news_api)

secret_name_alphavantage_api = "alphavantage-api-key"
alphavantage_api_key = client.get_secret(secret_name_alphavantage_api)

In [ ]:
import requests
import json

# Get API keys from key vault
ALPHAVANTAGE_API_KEY = alphavantage_api_key.value
NEWSAPI_API_KEY = news_api_key.value

def get_stock_ticker(company_name, api_key=ALPHAVANTAGE_API_KEY):
    """
    Fetches potential stock tickers for a given company name using the Alpha Vantage API.

    Args:
        company_name (str): The name of the company to search for.
        api_key (str, optional): Your Alpha Vantage API key. Defaults to ALPHAVANTAGE_API_KEY.

    Returns:
        list: A list of dictionaries, where each dictionary contains information
              about a matching ticker (e.g., 'symbol', 'name', 'region', 'currency').
              Returns None if there's an error or no results.
    """
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": api_key
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()
        if "bestMatches" in data:
            return data["bestMatches"]
        else:
            print(f"No matching tickers found for '{company_name}' or an error occurred: {data.get('Note', 'No detailed error message')}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching ticker: {e}")
        return None

def get_stock_news(ticker, api_key=NEWSAPI_API_KEY):
    """
    Fetches recent news articles for a given stock ticker using the NewsAPI.

    Args:
        ticker (str): The stock ticker symbol (e.g., AAPL).
        api_key (str, optional): Your NewsAPI API key. Defaults to NEWSAPI_API_KEY.

    Returns:
        list: A list of news article dictionaries (e.g., 'title', 'description', 'url', 'publishedAt', 'source').
              Returns None if there's an error or no results.
    """
    base_url = "https://newsapi.org/v2/everything"
    params = {
        "q": ticker,
        "apiKey": api_key,
        "sortBy": "publishedAt",
        "language": "en" # You can adjust the language if needed
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if data["status"] == "ok" and data["totalResults"] > 0:
            return data["articles"]
        else:
            print(f"No news found for ticker '{ticker}' or an error occurred: {data.get('message', 'No detailed error message')}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
        return None

In [25]:

def get_ticker_from_companyname(company_to_search):

    tickers = get_stock_ticker(company_to_search.strip())
    if tickers:
        try:
            selected_ticker = tickers[0]['1. symbol']
            return selected_ticker,tickers
        except Exception as e:
            print(f"The error in fetching the ticker :{e}")
            return None
    else:
        print(f"The stock code of the {company_to_search} is not found!")
        return None




In [30]:
user_provided_company = "Amazon "
selected_ticker = get_ticker_from_companyname(user_provided_company)

In [31]:
# news_articles = get_stock_news('MSFT')
# ls = []
# for _,v in enumerate(news_articles):
#     ls.append(v['description'])
# appended_desc = ". ".join([k for k in ls if k is not None])

In [32]:
def stock_news_extraction(ticker: str):

    try:

        news_articles = get_stock_news(ticker)
        ls = []
        for _,v in enumerate(news_articles):
            ls.append(v['description'])
        appended_desc = ". ".join([k for k in ls if k is not None])
        return appended_desc
    except Exception as e:
        print(f"The error is: {e} in extracting news atricle extraction")
        return None

  

In [33]:
latest_news = stock_news_extraction(selected_ticker)

In [34]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(api_version="2024-12-01-preview",model='gpt-4o-mini')

In [59]:
from langfuse.openai import AzureOpenAI
from langfuse import Langfuse
import langfuse as lf
import os, textwrap

import os
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-9fd0"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" #"http://localhost:3000"
os.environ['AZURE_OPENAI_ENDPOINT'] = ""
os.environ['AZURE_OPENAI_API_KEY'] = ""

lf = Langfuse(public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
              secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
              host=os.getenv("LANGFUSE_HOST", "http://localhost:3000"))

In [60]:
def log_prompt_langfuse(prompt):

    prompt_text = textwrap.dedent(prompt)

    prompt_v1 = lf.create_prompt(
        name="latest-news-extractor",
        prompt=prompt_text,
        config={"model":"gpt-4o-mini","temperature":0.2},
        labels=["production"],

    )
    return prompt_v1


In [49]:
# format = {"companyname":"", "stock_code":"", "newsdesc":"", "sentiment":"",
#           "people_names":[], "places_names":[], "other_companies_referred":[],
#           "related_industries":[], "market_implications":[], "confidence_score":0.0}

In [39]:
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

In [37]:
class StockAnalysis(BaseModel):
    """ Pydantic class to parse stock news description """
    companyname: str = Field(description="Name of the primary company discussed in the news")
    stock_code: str = Field(description="Stock ticker symbol of the primary company")
    newsdesc: str = Field(description="Brief description or summary of the news article")
    sentiment: str = Field(description="Overall sentiment expressed in the news (e.g., positive, negative, neutral)")
    people_names: List[str] = Field(default_factory=list, description="List of names of people mentioned in the news")
    places_names: List[str] = Field(default_factory=list, description="List of names of places mentioned in the news")
    other_companies_referred: List[str] = Field(default_factory=list, description="List of names of other companies referred to in the news")
    related_industries: List[str] = Field(default_factory=list, description="List of industries related to the news")
    market_implications: List[str] = Field(default_factory=list, description="Potential implications of the news on the market")
    confidence_score: float = Field(description="Confidence score (between 0.0 and 1.0) indicating the certainty of the extracted information")



In [38]:
prompt_template = """
    Analyze the following news excerpt and extract the relevant information in the following format:
    {format_instructions}

    News Excerpt:
    {news_excerpt}
    """

In [ ]:
def getFormattedResponse(news_desc: str):

    stock_news_parser = PydanticOutputParser(pydantic_object=StockAnalysis)    

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["news_excerpt"],
        partial_variables={"format_instructions": stock_news_parser.get_format_instructions()}
    )
    
    prompt_v1_logged = log_prompt_langfuse(prompt)
    llm_input = prompt.format_prompt(news_excerpt=news_desc)
    output = model(llm_input.to_messages())

    try:
        
        output = model(llm_input.to_messages())
        if hasattr(output, 'content'):
            parsed_output = stock_news_parser.parse(output.content)            
            return json.loads(parsed_output.model_dump_json())
            
        else:
            print(f"Error: Unexpected output format. No 'content' attribute found in the output: {output}")
            return None


    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your language model is generating output that conforms to the specified format.")
        print(f"Format Instructions: {stock_news_parser.get_format_instructions()}")
        return None


In [42]:
parsed_op = getFormattedResponse(latest_news)

In [ ]:
from langchain_core.runnables import RunnableLambda

runnable_getnews = RunnableLambda(stock_news_extraction)
runnable_parsedresponse = RunnableLambda(getFormattedResponse)

chain = runnable_getnews | runnable_parsedresponse

In [ ]:

result = chain.invoke(selected_ticker)


companyname='Amazon.com, Inc.' stock_code='AMZN' newsdesc='According to Bloomberg, Jeff Bezos, co-founder of Amazon, is a distant second at $227 billion. MGO One Seven LLC increased its holdings in shares of Amazon.com, Inc. by 12.7% during the fourth quarter, according to its most recent Form 13F filing with the SEC. Other institutional investors have also made significant adjustments to their holdings in the company, with notable increases and decreases reported.' sentiment='positive' people_names=['Elon Musk', 'Jeff Bezos', 'Thomas H. Kean Jr.'] places_names=['Bay Area', 'Wyoming'] other_companies_referred=['Meta', 'FedEx'] related_industries=['e-commerce', 'technology', 'entertainment'] market_implications=['Potential bullish sentiment for Amazon stock due to increased institutional investments and a recovery in personal fortunes of key figures in the industry.'] confidence_score=0.85


In [56]:
final_result = json.loads(result.json())

/tmp/ipykernel_188952/286763424.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  final_result = json.loads(result.json())


In [57]:
final_result

{'companyname': 'Amazon.com, Inc.',
 'stock_code': 'AMZN',
 'newsdesc': 'According to Bloomberg, Jeff Bezos, co-founder of Amazon, is a distant second at $227 billion. MGO One Seven LLC increased its holdings in shares of Amazon.com, Inc. by 12.7% during the fourth quarter, according to its most recent Form 13F filing with the SEC. Other institutional investors have also made significant adjustments to their holdings in the company, with notable increases and decreases reported.',
 'sentiment': 'positive',
 'people_names': ['Elon Musk', 'Jeff Bezos', 'Thomas H. Kean Jr.'],
 'places_names': ['Bay Area', 'Wyoming'],
 'other_companies_referred': ['Meta', 'FedEx'],
 'related_industries': ['e-commerce', 'technology', 'entertainment'],
 'market_implications': ['Potential bullish sentiment for Amazon stock due to increased institutional investments and a recovery in personal fortunes of key figures in the industry.'],
 'confidence_score': 0.85}